## Download/Import Dependencies

In [ ]:
!python --version
# !pip list -v | grep google-colab

Python 3.7.13


In [ ]:
# ### Install these when running for the first time  ######
!pip install contractions
!pip install mistletoe
!pip install html2text

!pip install -U -q PyDrive
!pip install google-colab

import nltk
nltk.download('stopwords')  
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

!pip install --upgrade tensorflow-gpu
!pip install tensorflow-hub
!pip install sentence-transformers #install sentence transformer models

!pip install umap-learn
!pip install hdbscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 7.3 MB/s 
     |████████████████████████████████| 106 kB 78.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 5.7 kB/s 
     |████████████████████████████████| 1.6 MB 56.5 MB/s 
     |████████████████████████████████| 5.8 MB 59.1 MB/s 
     |████████████████████████████████| 438 kB 70.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.

In [ ]:
import re, string, unicodedata    
import contractions   
from bs4 import BeautifulSoup

from mistletoe import markdown
from html2text import HTML2Text

import numpy as np
import pandas as pd 

from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import matplotlib.pyplot as plt 
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

#to import files from google drive
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive
from google.colab import auth 
from oauth2client.client import GoogleCredentials
from google.colab import drive

import tensorflow as tf
import tensorflow_hub as hub

from sentence_transformers import SentenceTransformer, util

import difflib
import webbrowser

from datetime import datetime as dt
import time

import copy

import umap.umap_ as umap
import hdbscan

In [ ]:
# Runtime GPU info
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

In [ ]:
# Runtime Memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


## Compress and upload the data to  Google Drive and Import the data from there

In [ ]:
# 1. Authorize Google Drive (required only for the first time, then comment it)
drive.mount('/content/gdrive', force_remount=True)

# 2. Compress the CSV files to .rar and upload the .rar file on your Google Drive (If compressing to .zip use zip commands instead)

# 3. Unrar the .rar file and put in the path after '-d' below
# !unrar x '/content/gdrive/My Drive/Colab Notebooks/ClassSpecsBLS.rar' -d '/content/gdrive/My Drive/Colab Notebooks/'
# !unrar x '/content/gdrive/My Drive/Colab Notebooks/pair-master.rar' -d '/content/gdrive/My Drive/Colab Notebooks/'

# 4. Read the CSV file from Google Drive saved in the above path
# df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/ClassSpecsBLS.csv')
# df.shape

Mounted at /content/gdrive


In [ ]:
# df.head()

In [ ]:
df.columns

Index(['ClassSpecID', 'EmployerID', 'Class_Title', 'Class_Concept',
       'ExamplesOfDuties', 'MinimumQualifications', 'SupplementalInfo',
       'OtherRequirements', 'OtherTitle1', 'OtherTitle2', 'BLS_Major',
       'BLS_Minor', 'BLS_Broad', 'BLS_Detailed'],
      dtype='object')

## Subsetting the data for analysis: 
Include Jobs which have clear seniority eg. Nurse-1,2,3; Lawyer-1,2,3 etc.

In [ ]:
# #Remove trailing and leading spaces from strings
# df_obj = df.select_dtypes(['object'])
# df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [ ]:
#subset some class titles
df2 = df.loc[df.Class_Title.str.contains("Assistant Caseworker|Community Health Nurse|Nurse Practitioner|Attorney Investigator", na=False)]
# df2.to_csv("test1.csv")
print(df2.shape)
df2.head()
# type(df.Class_Title[0])

(494, 14)


,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed
2358,3243,168,Assistant Caseworker II,To assess the social service needs of elderly ...,"Attract, interview and screen potential home h...","<b><u>Knowledge of:</b></u> English usage, sp...",NaN,NaN,NaN,NaN,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,Home Health and Personal Care Aides,Home Health Aides
2360,3245,168,Assistant Caseworker I,To assess the social service needs of elderly ...,"Attract, interview and screen potential home h...","<b><u>Knowledge of:</b></u> English usage, sp...",NaN,NaN,NaN,NaN,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,Home Health and Personal Care Aides,Home Health Aides
2419,3308,168,Nurse Practitioner,To perform physical screening assessments; to ...,"<b><u>DUTIES</b></u> may include, but are not ...",<b><u>NECESSARY EMPLOYMENT STANDARDS Knowledg...,NaN,NaN,NaN,NaN,Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners
3571,4916,141,Family Nurse Practitioner,<strong>Bargaining Unit: CNA - Registered Nurs...,Positions in this class provide in-home or out...,<strong>License Required:</strong> Possession ...,<strong>Knowledge of:</strong><ul>\t<li>The me...,<strong>Under medical direction and consultati...,Established: August 1975<br>Revised: September...,NaN,Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners
4394,5875,385,Nurse Practitioner,"Under general supervision, provide health care...","<strong>Duties may include, but are not limite...",<strong>Note: </strong>The level and scope of ...,NaN,NaN,Nurse Practitioner Trainee-Nurse Practitioner,"Established: 10/26/1978<br>Revised: 8/29/2005,...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners


In [ ]:
# sample_titles = ['Assistant Caseworker I','Assistant Caseworker II', 'Assistant Caseworker III',\
#                 #  'Community Health Nurse 3A', 'Community Health Nurse 2A', 'Community Health Nurse 1A',\
#                  'Nurse Practitioner I','Nurse Practitioner II',\
#                  "State's Attorney Investigator I", "State's Attorney Investigator II", "State's Attorney Investigator III"]

# sample_BLS_Detailed = ['Information and Record Clerks, All Other', 'Nurse Practitioners', 'Home Health Aides', 'Lawyers', 'Registered Nurses']
# # df['Class_Title'] = df['Class_Title'].apply(lambda x: str(x).strip())
# df2 = df[df['BLS_Detailed'].isin(sample_BLS_Detailed)]
# print(df2.shape)
# df2.groupby(['BLS_Detailed']).size()
# df2.head()

(15646, 14)


BLS_Detailed
Home Health Aides                            295
Information and Record Clerks, All Other    3545
Lawyers                                     8164
Nurse Practitioners                          636
Registered Nurses                           3006
dtype: int64

In [ ]:
# describing stats of variables
df2.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ClassSpecID,494.0,NaN,NaN,NaN,911884.609312,430255.368413,3243.0,770658.25,995667.0,1238149.75,1440204.0
EmployerID,494.0,NaN,NaN,NaN,2520.587045,1890.529268,141.0,1470.0,1923.0,2883.0,11263.0
Class_Title,494,272,Nurse Practitioner,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Class_Concept,439,419,<div>The purpose of this position is to serve ...,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ExamplesOfDuties,420,406,A Job Specification is not maintained for this...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MinimumQualifications,404,389,<ul>\t<li>Valid CPR Certification\t<li>Must po...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SupplementalInfo,363,322,"<p style=""margin:0in;"">2 - Limited supervision...",7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OtherRequirements,207,186,<div>Current licensure as a registered nurse i...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OtherTitle1,173,155,&nbsp;,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OtherTitle2,130,105,None,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Count of unique values
# df2.Class_Title.nunique()
df2.ClassSpecID.nunique()

#check for nulls
df2.isnull().sum(axis=0)     

ClassSpecID                0
EmployerID                 0
Class_Title                0
Class_Concept             55
ExamplesOfDuties          74
MinimumQualifications     90
SupplementalInfo         131
OtherRequirements        287
OtherTitle1              321
OtherTitle2              364
BLS_Major                 55
BLS_Minor                 55
BLS_Broad                 55
BLS_Detailed              55
dtype: int64

## Data Pre-processing:

In [ ]:
df2.fillna('', inplace=True)

In [ ]:
df2['New_JD'] = df2['Class_Concept'] + "<br><br>" + df2['ExamplesOfDuties'] + "<br><br>" + df2['MinimumQualifications'] + "<br><br>" + df2['SupplementalInfo']+ "<br><br>" + df2['OtherRequirements'] + "<br><br>" + df2['OtherTitle1'] + "<br><br>" + df2['OtherTitle2']
df2.head()

,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed,New_JD
2358,3243,168,Assistant Caseworker II,To assess the social service needs of elderly ...,"Attract, interview and screen potential home h...","<b><u>Knowledge of:</b></u> English usage, sp...",,,,,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,Home Health and Personal Care Aides,Home Health Aides,To assess the social service needs of elderly ...
2360,3245,168,Assistant Caseworker I,To assess the social service needs of elderly ...,"Attract, interview and screen potential home h...","<b><u>Knowledge of:</b></u> English usage, sp...",,,,,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,Home Health and Personal Care Aides,Home Health Aides,To assess the social service needs of elderly ...
2419,3308,168,Nurse Practitioner,To perform physical screening assessments; to ...,"<b><u>DUTIES</b></u> may include, but are not ...",<b><u>NECESSARY EMPLOYMENT STANDARDS Knowledg...,,,,,Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,To perform physical screening assessments; to ...
3571,4916,141,Family Nurse Practitioner,<strong>Bargaining Unit: CNA - Registered Nurs...,Positions in this class provide in-home or out...,<strong>License Required:</strong> Possession ...,<strong>Knowledge of:</strong><ul>\t<li>The me...,<strong>Under medical direction and consultati...,Established: August 1975<br>Revised: September...,,Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,<strong>Bargaining Unit: CNA - Registered Nurs...
4394,5875,385,Nurse Practitioner,"Under general supervision, provide health care...","<strong>Duties may include, but are not limite...",<strong>Note: </strong>The level and scope of ...,,,Nurse Practitioner Trainee-Nurse Practitioner,"Established: 10/26/1978<br>Revised: 8/29/2005,...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,"Under general supervision, provide health care..."


In [ ]:
data = df2.copy()

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

data['Class_Concept'] = data['Class_Concept'].apply(lambda x: strip_html(str(x)))
data['ExamplesOfDuties'] = data['ExamplesOfDuties'].apply(lambda x: strip_html(str(x)))
data['MinimumQualifications'] = data['MinimumQualifications'].apply(lambda x: strip_html(str(x)))
data['SupplementalInfo'] = data['SupplementalInfo'].apply(lambda x: strip_html(str(x)))
data['OtherRequirements'] = data['OtherRequirements'].apply(lambda x: strip_html(str(x)))
data['OtherTitle1'] = data['OtherTitle1'].apply(lambda x: strip_html(str(x)))
data['OtherTitle2'] = data['OtherTitle2'].apply(lambda x: strip_html(str(x)))

#Remove all the bullets in the text
# data['Class_Concept'] = data['Class_Concept'].apply(lambda x: str(x).replace('•', ''))
# data['ExamplesOfDuties'] = data['ExamplesOfDuties'].apply(lambda x: str(x).replace('•', ''))
# data['MinimumQualifications'] = data['ExamplesOfDuties'].apply(lambda x: str(x).replace('•', ''))
# data['ExamplesOfDuties'] = data['ExamplesOfDuties'].apply(lambda x: str(x).replace('•', ''))
# data['ExamplesOfDuties'] = data['ExamplesOfDuties'].apply(lambda x: str(x).replace('•', ''))
# data['OtherTitle1'] = data['OtherTitle1'].apply(lambda x: str(x).replace('•', ''))
# data['OtherTitle2'] = data['OtherTitle2'].apply(lambda x: str(x).replace('•', ''))

#Create Job Description column by merging Class_Concept, ExamplesOfDuties, MinimumQualifications, SupplementalInfo, OtherRequirements columns
data['Job_Desc'] = data[data.columns[3:9]].apply(lambda x: '. '.join(x.dropna().astype(str)), axis=1)

data.head()

,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed,Job_Desc
2358,3243,168,Assistant Caseworker II,To assess the social service needs of elderly ...,"Attract, interview and screen potential home h...","Knowledge of: English usage, spelling, gramma...",nan,nan,nan,nan,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,Home Health and Personal Care Aides,Home Health Aides,To assess the social service needs of elderly ...
2360,3245,168,Assistant Caseworker I,To assess the social service needs of elderly ...,"Attract, interview and screen potential home h...","Knowledge of: English usage, spelling, gramma...",nan,nan,nan,nan,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,Home Health and Personal Care Aides,Home Health Aides,To assess the social service needs of elderly ...
2419,3308,168,Nurse Practitioner,To perform physical screening assessments; to ...,"DUTIES may include, but are not necessarily li...",NECESSARY EMPLOYMENT STANDARDS Knowledge of: ...,nan,nan,nan,nan,Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,To perform physical screening assessments; to ...
3571,4916,141,Family Nurse Practitioner,Bargaining Unit: CNA - Registered Nurses Unit ...,Positions in this class provide in-home or out...,License Required: Possession of a valid licens...,"Knowledge of: The medical, biological and phys...","Under medical direction and consultation, does...",Established: August 1975Revised: September 200...,nan,Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,Bargaining Unit: CNA - Registered Nurses Unit ...
4394,5875,385,Nurse Practitioner,"Under general supervision, provide health care...","Duties may include, but are not limited to, th...",Note: The level and scope of the knowledge and...,nan,nan,Nurse Practitioner Trainee-Nurse Practitioner,"Established: 10/26/1978Revised: 8/29/2005, 9/0...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,"Under general supervision, provide health care..."


In [ ]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

data['Job_Desc'] = data['Job_Desc'].apply(lambda x: replace_contractions(x))
# data.to_csv("/content/gdrive/My Drive/Colab Notebooks/Nurse_Assistants_Raw_JD.csv")
data.head()

,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed,Job_Desc
2358,3243,168,Assistant Caseworker II,To assess the social service needs of elderly ...,"Attract, interview and screen potential home h...","Knowledge of: English usage, spelling, gramma...",nan,nan,nan,nan,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,Home Health and Personal Care Aides,Home Health Aides,To assess the social service needs of elderly ...
2360,3245,168,Assistant Caseworker I,To assess the social service needs of elderly ...,"Attract, interview and screen potential home h...","Knowledge of: English usage, spelling, gramma...",nan,nan,nan,nan,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,Home Health and Personal Care Aides,Home Health Aides,To assess the social service needs of elderly ...
2419,3308,168,Nurse Practitioner,To perform physical screening assessments; to ...,"DUTIES may include, but are not necessarily li...",NECESSARY EMPLOYMENT STANDARDS Knowledge of: ...,nan,nan,nan,nan,Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,To perform physical screening assessments; to ...
3571,4916,141,Family Nurse Practitioner,Bargaining Unit: CNA - Registered Nurses Unit ...,Positions in this class provide in-home or out...,License Required: Possession of a valid licens...,"Knowledge of: The medical, biological and phys...","Under medical direction and consultation, does...",Established: August 1975Revised: September 200...,nan,Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,Bargaining Unit: CNA - Registered Nurses Unit ...
4394,5875,385,Nurse Practitioner,"Under general supervision, provide health care...","Duties may include, but are not limited to, th...",Note: The level and scope of the knowledge and...,nan,nan,Nurse Practitioner Trainee-Nurse Practitioner,"Established: 10/26/1978Revised: 8/29/2005, 9/0...",Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,"Under general supervision, provide health care..."


# Create Pair-wise Master data

In [ ]:
# Subset the data
data2 = data[['ClassSpecID', 'EmployerID', 'Class_Title', 'Job_Desc','BLS_Detailed']]
data2.fillna('', inplace=True)
data2.dropna(how='all', axis=1, inplace=True)
print(data2.shape)
data2.head()

In [ ]:
#performs a cross-join
pair_master = pd.merge(data2, data2, how = 'left', left_on = ['EmployerID', 'BLS_Detailed'], right_on = ['EmployerID', 'BLS_Detailed'])
# pd.merge(final, data[['ClassSpecID', 'Cluster_ID']], how = 'left', left_on = 'ClassSpecID1', right_on = 'ClassSpecID')

# #rename columns
pair_master2 = pair_master.rename(columns={'ClassSpecID_x':'ClassSpecID1','ClassSpecID_y':'ClassSpecID2','Class_Title_x':'Class_Title1','Class_Title_y':'Class_Title2','EmployerID_x':'EmployerID1','EmployerID_y':'EmployerID2',\
        'Job_Desc_x':'Job_Desc1','Job_Desc_y':'Job_Desc2','BLS_Detailed_x':'BLS_Detailed1','BLS_Detailed_y':'BLS_Detailed2'})

# #rearrange columns and add pair_ID
pair_master2['Pair_ID'] = pair_master2['ClassSpecID1'].astype(str) + '-' + pair_master2['ClassSpecID2'].astype(str)
col_list = ['Pair_ID','ClassSpecID1','ClassSpecID2','Class_Title1','Class_Title2','EmployerID','BLS_Detailed','Job_Desc1','Job_Desc2']
pair_master2 = pair_master2[col_list]
pair_master2.to_csv('/content/gdrive/My Drive/Colab Notebooks/pair-master.csv')
print(pair_master2.shape)
pair_master2.head()

(1824, 9)


,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,EmployerID,BLS_Detailed,Job_Desc1,Job_Desc2
0,3243-3243,3243,3243,Assistant Caseworker II,Assistant Caseworker II,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...
1,3243-3245,3243,3245,Assistant Caseworker II,Assistant Caseworker I,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...
2,3243-1094664,3243,1094664,Assistant Caseworker II,Assistant Caseworker III,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...
3,3245-3243,3245,3243,Assistant Caseworker I,Assistant Caseworker II,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...
4,3245-3245,3245,3245,Assistant Caseworker I,Assistant Caseworker I,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...


In [ ]:
pair_master2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/pair-master.csv')
print(pair_master2.shape)
# sample_titles = ['Community Health Assistant','Medical Assistant II', 'NURSING ASSISTANT','Nursing Supervisor']
    
# pair_master_5k = pair_master2[pair_master2['Class_Title1'].isin(sample_titles)]
pair_master3 = pair_master2.copy()

pair_master3['key'] = pair_master3.apply(lambda row: tuple(sorted((row['ClassSpecID1'], row['ClassSpecID2']))), axis=1)
pair_master3.drop_duplicates(subset=['key'], keep='first', inplace=True, ignore_index=False)
print(pair_master3.shape)
pair_master3.head()

(1824, 10)
(1159, 11)


,Unnamed: 0,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,EmployerID,BLS_Detailed,Job_Desc1,Job_Desc2,key
0,0,3243-3243,3243,3243,Assistant Caseworker II,Assistant Caseworker II,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3243, 3243)"
1,1,3243-3245,3243,3245,Assistant Caseworker II,Assistant Caseworker I,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3243, 3245)"
2,2,3243-1094664,3243,1094664,Assistant Caseworker II,Assistant Caseworker III,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3243, 1094664)"
4,4,3245-3245,3245,3245,Assistant Caseworker I,Assistant Caseworker I,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3245, 3245)"
5,5,3245-1094664,3245,1094664,Assistant Caseworker I,Assistant Caseworker III,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3245, 1094664)"


# Estimate Similarity Scores

# 1. Universal Sentence Encoder (USE)

### 1.1 Load Universal Sentence Encoder (USE) model from TF-Hub

In [ ]:
# pair_master = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/test.csv')

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model_USE = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


### 1.2. Compute the similarity between two job descriptions:

In [ ]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def cosine_similarity_USE(row):
    try:
        sentence1 = row.Job_Desc1
        sentence2 = row.Job_Desc2
        return round(float(cosine(model_USE([sentence1])[0], model_USE([sentence2])[0]) * 100), 2)
    except ValueError:
        return 0

def cosine_similarity_USE_Title(row):
    try:
        sentence1 = row.Class_Title1
        sentence2 = row.Class_Title2
        return round(float(cosine(model_USE([sentence1])[0], model_USE([sentence2])[0]) * 100), 2)
    except ValueError:
        return 0

In [ ]:
start_time = dt.now()
pair_master3["USE_sim"] = pair_master3.apply(cosine_similarity_USE, axis=1)
end_time = dt.now()
difference = end_time - start_time
print(f" Total time taken is {difference}")
pair_master3.head()

 Total time taken is 0:00:17.360558


,Unnamed: 0,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,EmployerID,BLS_Detailed,Job_Desc1,Job_Desc2,key,USE_sim
0,0,3243-3243,3243,3243,Assistant Caseworker II,Assistant Caseworker II,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3243, 3243)",100.00
1,1,3243-3245,3243,3245,Assistant Caseworker II,Assistant Caseworker I,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3243, 3245)",97.72
2,2,3243-1094664,3243,1094664,Assistant Caseworker II,Assistant Caseworker III,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3243, 1094664)",93.77
4,4,3245-3245,3245,3245,Assistant Caseworker I,Assistant Caseworker I,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3245, 3245)",100.00
5,5,3245-1094664,3245,1094664,Assistant Caseworker I,Assistant Caseworker III,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3245, 1094664)",93.60


### 1.3. Compute the similarity between two job titles:

In [ ]:
start_time = dt.now()
pair_master3["USE_Title_sim"] = pair_master3.apply(cosine_similarity_USE_Title, axis=1)
end_time = dt.now()
difference = end_time - start_time
print(f" Total time taken is {difference}")
pair_master3.head()

 Total time taken is 0:00:11.282559


,Unnamed: 0,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,EmployerID,BLS_Detailed,Job_Desc1,Job_Desc2,key,USE_sim,USE_Title_sim
0,0,3243-3243,3243,3243,Assistant Caseworker II,Assistant Caseworker II,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3243, 3243)",100.00,100.00
1,1,3243-3245,3243,3245,Assistant Caseworker II,Assistant Caseworker I,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3243, 3245)",97.72,76.32
2,2,3243-1094664,3243,1094664,Assistant Caseworker II,Assistant Caseworker III,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3243, 1094664)",93.77,76.98
4,4,3245-3245,3245,3245,Assistant Caseworker I,Assistant Caseworker I,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3245, 3245)",100.00,100.00
5,5,3245-1094664,3245,1094664,Assistant Caseworker I,Assistant Caseworker III,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3245, 1094664)",93.60,77.38


In [ ]:
# import copy
# #creates a new dataframe
# new_df = pd.DataFrame() 

# for index, row in pair_master_5k.iterrows():
#     new_row = copy.deepcopy(row)
#     new_dict = new_row.to_dict()
    
#     new_dict['USE_sim'] = cosine(model([new_dict['Job_Desc1']])[0], model([new_dict['Job_Desc2']])[0])

#     new_df=new_df.append(pd.DataFrame(new_dict, index = [0])) #, ignore_index=True

# # new_df.to_csv('/content/gdrive/My Drive/Colab Notebooks/Data Output From Python.csv')
# new_df.head()
# type(list(pair_master_5k.iloc[0,7]))

.

# 2. SentenceTransformers (SBERT)
### 2.1 Load SentenceTransformers (SBERT) model 

In [ ]:
model_SBERT = SentenceTransformer("nli-distilroberta-base-v2")

### 2.2. Compute the similarity between two job descriptions::

In [ ]:
def cosine_similarity_SBERT(row):
    sentence1 = row.Job_Desc1
    sentence2 = row.Job_Desc2
    embedding1 = model_SBERT.encode([sentence1])
    embedding2 = model_SBERT.encode([sentence2])

    #Compute cosine similarity between all pairs
    return round(float(util.cos_sim(embedding1, embedding2)[0][0] * 100), 2) 


def cosine_similarity_SBERT_Title(row):
    sentence1 = row.Class_Title1
    sentence2 = row.Class_Title2
    embedding1 = model_SBERT.encode([sentence1])
    embedding2 = model_SBERT.encode([sentence2])

    #Compute cosine similarity between all pairs
    return round(float(util.cos_sim(embedding1, embedding2)[0][0] * 100), 2) 

In [ ]:
start_time = dt.now()
pair_master3["SBert_sim"] = pair_master3.apply(cosine_similarity_SBERT, axis=1)
end_time = dt.now()
difference = end_time - start_time
print(f" Total time taken is {difference}")

 Total time taken is 0:00:26.034465


### 2.3. Compute the similarity between two job titles:

In [ ]:
start_time = dt.now()
pair_master3["SBert_Title_sim"] = pair_master3.apply(cosine_similarity_SBERT_Title, axis=1)
end_time = dt.now()
difference = end_time - start_time
print(f" Total time taken is {difference}")

 Total time taken is 0:00:17.162317


In [ ]:
# # Drop rows with SBert_Title_sim < 80
# pair_master3 = pair_master3.drop(pair_master[pair_master3.SBert_Title_sim < 80].index)
# # Drop rows with SBert_sim < 60
# pair_master3 = pair_master3.drop(pair_master[pair_master3.SBert_sim < 60].index)
# # Drop rows with 'DO NOT USE' in title
# pair_master3 = pair_master3.loc[~pair_master3.Class_Title1.str.contains("DO NOT USE", na=False)]
pair_master3.head()

,Unnamed: 0,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,EmployerID,BLS_Detailed,Job_Desc1,Job_Desc2,key,USE_sim,USE_Title_sim,SBert_sim,SBert_Title_sim
0,0,3243-3243,3243,3243,Assistant Caseworker II,Assistant Caseworker II,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3243, 3243)",100.00,100.00,100.00,100.00
1,1,3243-3245,3243,3245,Assistant Caseworker II,Assistant Caseworker I,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3243, 3245)",97.72,76.32,94.79,92.40
2,2,3243-1094664,3243,1094664,Assistant Caseworker II,Assistant Caseworker III,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3243, 1094664)",93.77,76.98,95.18,89.82
4,4,3245-3245,3245,3245,Assistant Caseworker I,Assistant Caseworker I,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3245, 3245)",100.00,100.00,100.00,100.00
5,5,3245-1094664,3245,1094664,Assistant Caseworker I,Assistant Caseworker III,168,Home Health Aides,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,"(3245, 1094664)",93.60,77.38,92.68,88.46


In [ ]:
pair_master3.to_csv('/content/gdrive/My Drive/Colab Notebooks/Seniority_classification/Classification_Data.csv',index=False)

In [ ]:
pair_master3.shape

(1159, 15)

## Text Differences: extract seniority text

#### Method: difflib module

In [ ]:
import re

#creates a new dataframe
headers = columns = list(pair_master3)
final = pd.DataFrame() 


# pair_master2 = pair_master.copy()
# pair_master2 = pair_master[pair_master['Pair_ID']=='3245-1094664']
# pair_master2 = pair_master[~pair_master['Pair_ID'].isin(['1094664-3243'])]
# print(pair_master2.shape)

start_time = dt.now()

# Function that matches sentences 
#https://stackoverflow.com/questions/65702508/python-difflib-get-close-matches-comparing-modified-text-but-returning-original)
from difflib import SequenceMatcher, _nlargest  # necessary imports of functions used by modified get_close_matches

def get_close_matches_lower(word, possibilities, n=1, cutoff=0.70): 
  '''word: sentence to be matched from JD1(string)
     possibilities: list of sentences to be matched with in JD2
     n: sorts sentences on the basis of match accuracy and gives 1 result
     cutoff: match accuracy
  '''
  if not n >  0:
      raise ValueError("n must be > 0: %r" % (n,))
  if not 0.0 <= cutoff <= 1.0:
      raise ValueError("cutoff must be in [0.0, 1.0]: %r" % (cutoff,))
  result = []
  s = SequenceMatcher()
  s.set_seq2(word)
  for x in possibilities:
      s.set_seq1(x.lower())  # lower-case for comparison
      if s.real_quick_ratio() >= cutoff and \
          s.quick_ratio() >= cutoff and \
          s.ratio() >= cutoff:
          result.append((possibilities.index(x)+1, x))

  # Move the best scorers to head of list
  result = _nlargest(n, result)
  # Strip scores for the best n matches
  return result
  #[x for sent_id, x in result]

for index, row in pair_master3.iterrows():
  new_row = copy.deepcopy(row)
  # new_dict = new_row.to_dict
  
  #split the paragraph to sentences, and delete sentences of length<30 because no emglish sentence usually is less than that.
  from_lines = [x.strip() for x in re.split(r'[;:.]', new_row['Job_Desc1']) if len(x)>30]  
  to_lines = [x.strip() for x in re.split(r'[;:.]', new_row['Job_Desc2'])if len(x)>30]

  # print(len(from_lines))
  # print(len(to_lines))

  # print([get_close_matches_lower(x, to_lines) for x in from_lines])

  # Part-1: First round of matching (Get the sentences in JD-1 that match with JD-2)
  try:
    sent_ID1, from_lines1, senior_ID1, sent_ID2, to_lines1, senior_ID2 = zip(*[(from_lines.index(x)+1, x, 0, get_close_matches_lower(x, to_lines)[0][0], get_close_matches_lower(x, to_lines)[0][1], 0) for x in from_lines if len(get_close_matches_lower(x, to_lines))>0])
  except:
    sent_ID1, from_lines1, senior_ID1, sent_ID2, to_lines1, senior_ID2 = ([] for i in range(6)) #if above lists are empty due to no match, it throws an error

  # print(len(from_lines1))
  # print(len(to_lines1))
  # print(len(sent_ID1))
  # print(len(sent_ID2))

  # Create a new list by removing the matched sentences from the full list
  from_lines2 = [x for x in from_lines if x not in from_lines1]
  to_lines2 = [x for x in to_lines if x not in to_lines1]

  # print(len(from_lines2))
  # print(len(to_lines2))

  # Part-2: Second round of matching (Get the sentences in JD-2 that were not matched with JD-1 in the first round and try matching)
  try:
    sent_ID3, from_lines3, senior_ID3, sent_ID4, to_lines3, senior_ID4 = zip(*[(to_lines.index(x)+1, x, 0, get_close_matches_lower(x, from_lines)[0][0], get_close_matches_lower(x, from_lines)[0][1], 0) for x in to_lines2 if len(get_close_matches_lower(x, from_lines))>0])
  except:
    sent_ID3, from_lines3, senior_ID3, sent_ID4, to_lines3, senior_ID4 = ([] for i in range(6)) #if above lists are empty due to no match, it throws an error

  # Create a new list by removing the matched sentences from the non-matched list in the first round of matching
  from_lines4 = [x for x in from_lines2 if x not in from_lines3]
  sent_ID5 = [from_lines.index(x)+1 for x in from_lines4]
  to_lines4 = [x for x in to_lines2 if x not in to_lines3]
  sent_ID6 = [to_lines.index(x)+1 for x in to_lines4]

  # convert above lists into a pandas dataframe|
  data1 = pd.DataFrame(list(zip(sent_ID1, from_lines1, senior_ID1, sent_ID2, to_lines1, senior_ID2)),columns = ['sent_ID1', 'JD1', 'senior_ID1', 'sent_ID2', 'JD2', 'senior_ID2'])
  data2 = pd.DataFrame(list(zip(sent_ID3, from_lines3, senior_ID3, sent_ID4, to_lines3, senior_ID4)),columns = ['sent_ID1', 'JD1', 'senior_ID1', 'sent_ID2', 'JD2', 'senior_ID2'])
  data3 = pd.DataFrame({'sent_ID1': pd.Series(sent_ID5),'JD1': pd.Series(from_lines4), 'senior_ID1': pd.Series(['1']*len(from_lines4)), 'sent_ID2': pd.Series(['']*len(from_lines4)), 'JD2': pd.Series(['']*len(from_lines4)), 'senior_ID2': pd.Series(['']*len(from_lines4))})
  data4 = pd.DataFrame({'sent_ID1': pd.Series(['']*len(to_lines4)), 'JD1': pd.Series(['']*len(to_lines4)), 'senior_ID1': pd.Series(['']*len(to_lines4)), 'sent_ID2': pd.Series(sent_ID6), 'JD2': pd.Series(to_lines4), 'senior_ID2': pd.Series(['1']*len(to_lines4))})

  # join the sentences that did not match to create seniority texts
  data3.sort_values('sent_ID1')
  new_row['Seniority_text1'] = '. '.join(data3['JD1'])

  data4.sort_values('sent_ID2')
  new_row['Seniority_text2'] = '. '.join(data4['JD2'])

  new_row = new_row.to_frame().T

  final = final.append(new_row, ignore_index=True)

col_list1 = ['Pair_ID','ClassSpecID1','ClassSpecID2','Class_Title1','Class_Title2','EmployerID','Job_Desc1','Job_Desc2', 'Seniority_text1','Seniority_text2','USE_sim','USE_Title_sim','SBert_sim','SBert_Title_sim']
final = final[col_list1]

final.to_csv('/content/gdrive/My Drive/Colab Notebooks/Seniority_classification/Final.csv', index=False)

end_time = dt.now()
difference = end_time - start_time
print(f" Total time taken is {difference}")

final.head()

 Total time taken is 0:20:03.819677


,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,EmployerID,Job_Desc1,Job_Desc2,Seniority_text1,Seniority_text2,USE_sim,USE_Title_sim,SBert_sim,SBert_Title_sim
0,3243-3243,3243,3243,Assistant Caseworker II,Assistant Caseworker II,168,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,DISTINGUISHING CHARACTERISTICS This is the jou...,DISTINGUISHING CHARACTERISTICS This is the jou...,100.0,100.0,100.0,100.0
1,3243-3245,3243,3245,Assistant Caseworker II,Assistant Caseworker I,168,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,DISTINGUISHING CHARACTERISTICS This is the jou...,DISTINGUISHING CHARACTERISTICS This is an entr...,97.72,76.32,94.79,92.4
2,3243-1094664,3243,1094664,Assistant Caseworker II,Assistant Caseworker III,168,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,DISTINGUISHING CHARACTERISTICS This is the jou...,DISTINGUISHING CHARACTERISTICSThis is an advan...,93.77,76.98,95.18,89.82
3,3245-3245,3245,3245,Assistant Caseworker I,Assistant Caseworker I,168,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,,determine client's initial social service need...,100.0,100.0,100.0,100.0
4,3245-1094664,3245,1094664,Assistant Caseworker I,Assistant Caseworker III,168,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,DISTINGUISHING CHARACTERISTICS This is an entr...,DISTINGUISHING CHARACTERISTICSThis is an advan...,93.6,77.38,92.68,88.46


## Clustering the jobs based on Job Descriptions

Reference: https://towardsdatascience.com/clustering-sentence-embeddings-to-identify-intents-in-short-text-48d22d3bf02e

## 4.1. Function to extract embeddings from the models

In [ ]:
def embed(model, model_type, sentences):
  """
  wrapper function for generating message embeddings
  """
  
  if model_type == 'use':
      embeddings = model(sentences)
  elif model_type == 'sentence transformer':
      embeddings = model.encode(sentences)
  
  return embeddings


JD_text = data['New_JD'].tolist() #change dataset name

# generate embeddings for each model
embeddings_USE = embed(model_USE, 'use', JD_text)
embeddings_SBERT = embed(model_SBERT, 'sentence transformer', JD_text)

In [ ]:
embeddings = [embeddings_USE, embeddings_SBERT]

for emb in embeddings:
  print(emb.shape)

(254, 512)
(254, 768)


## 4.2. Dimensionality Reduction using Uniform Manifold Approximation and Projection (UMAP)

Note-1: UMAP introduced in 2020 is much faster and more scalable than t-SNE

Note-2: Our sentence embeddings have high dimensionality (>500 features each). Curse of Dimensionality is that distance measures have little to no meaning. So dimensionality reduction is done before clustering to improve the results significantly.

Note-3: Density-based algorithms are a good option here as they do not require specifying the number of clusters and are indifferent to cluster shape. So we use Hierarchical Density-Based Spatial Clustering of Applications with Noise (HDBSCAN).

### 4.2.1. Function to Reduce dimensions and generate clusters

In [ ]:
def generate_clusters(message_embeddings,
                      n_neighbors,
                      n_components, 
                      min_cluster_size,
                      random_state = None):
    """
    Generate HDBSCAN cluster object after reducing embedding dimensionality with UMAP.
    The n_neighbors parameter controls the size of the neighborhood UMAP looks to learn the manifold structure, and so lower values of n_neighbors will focus more on the very local structure.
    The n_components parameter controls the dimensionality of the final embedded data after performing dimensionality reduction on the input data.
    In Angelov’s Top2Vec paper he mentions that n_neighbors = 15 and n_components = 5 worked best for his downstream tasks, but it is unlikely this would always be the case for any dataset.
    """
    
    umap_embeddings = (umap.UMAP(n_neighbors=n_neighbors, 
                                n_components=n_components, 
                                metric='cosine', 
                                random_state=random_state)
                            .fit_transform(message_embeddings))

    clusters = hdbscan.HDBSCAN(min_cluster_size = min_cluster_size,
                               metric='euclidean', 
                               cluster_selection_method='eom').fit(umap_embeddings)

    return clusters

In [ ]:
clusters_USE = generate_clusters(embeddings_USE, 15, 5, 5, random_state = None)

In [ ]:
# help(clusters_USE)
cluster_ID = clusters_USE.labels_.tolist()
# print(cluster_ID)
# clusters_USE.probabilities_
# print(len(set(clusters_USE.labels_)))
# Add cluster ID column to 'Data' dataset
data['Cluster_ID'] = cluster_ID
data.head()

,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed,New_JD,Cluster_ID
2358,3243,168,Assistant Caseworker II,To assess the social service needs of elderly ...,"Attract, interview and screen potential home h...","Knowledge of:. English usage, spelling, gramma...",nan,nan,nan,nan,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,Home Health and Personal Care Aides,Home Health Aides,To assess the social service needs of elderly ...,0
2360,3245,168,Assistant Caseworker I,To assess the social service needs of elderly ...,"Attract, interview and screen potential home h...","Knowledge of:. English usage, spelling, gramma...",nan,nan,nan,nan,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,Home Health and Personal Care Aides,Home Health Aides,To assess the social service needs of elderly ...,0
2419,3308,168,Nurse Practitioner,To perform physical screening assessments; to ...,"DUTIES. may include, but are not necessarily l...",NECESSARY EMPLOYMENT STANDARDS Knowledge of:....,nan,nan,nan,nan,Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,To perform physical screening assessments; to ...,0
3571,4916,141,Family Nurse Practitioner,Bargaining Unit: CNA - Registered Nurses Unit....,Positions in this class provide in-home or out...,License Required:. Possession of a valid licen...,"Knowledge of:. The medical, biological and phy...","Under medical direction and consultation, does...",Established: August 1975. Revised: September 2...,nan,Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,Bargaining Unit: CNA - Registered Nurses Unit....,0
4395,5876,385,Nurse Practitioner Trainee,"Under general supervision, provide primary hea...","Duties may include, but are not limited to, th...",Note:. The level and scope of the knowledge an...,nan,nan,nan,nan,Healthcare Practitioners and Technical Occupat...,Healthcare Diagnosing or Treating Practitioners,Nurse Practitioners,Nurse Practitioners,"Under general supervision, provide primary hea...",0


## Extract Cluster_ID column to Final dataset: perform a left-join

In [ ]:
final2 = pd.merge(final, data[['ClassSpecID', 'Cluster_ID']], how = 'left', left_on = 'ClassSpecID1', right_on = 'ClassSpecID')
final2 = final2.rename(columns={'ClassSpecID':'ClassSpecID_x','Cluster_ID':'Cluster_ID1'}) #rename columns
final2 = pd.merge(final2, data[['ClassSpecID', 'Cluster_ID']], how = 'left', left_on = 'ClassSpecID2', right_on = 'ClassSpecID')
final2 = final2.rename(columns={'ClassSpecID':'ClassSpecID_y','Cluster_ID':'Cluster_ID2'}) #rename columns
final2.head()

,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,EmployerID,Job_Desc1,Job_Desc2,USE_sim,USE_Title_sim,SBert_sim,SBert_Title_sim,Seniority_text1,Seniority_text2,ClassSpecID_x,Cluster_ID1,ClassSpecID_y,Cluster_ID2
0,3243-3245,3243,3245,Assistant Caseworker II,Assistant Caseworker I,168,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,97.63,76.32,93.71,92.4,DISTINGUISHING CHARACTERISTICS This is the jou...,DISTINGUISHING CHARACTERISTICS This is an entr...,3243,0,3245,0
1,3243-1094664,3243,1094664,Assistant Caseworker II,Assistant Caseworker III,168,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,94.68,76.98,94.93,89.82,DISTINGUISHING CHARACTERISTICS This is the jou...,DISTINGUISHING CHARACTERISTICS. This is an adv...,3243,0,1094664,0
2,3245-1094664,3245,1094664,Assistant Caseworker I,Assistant Caseworker III,168,To assess the social service needs of elderly ...,To assess the social service needs of elderly ...,94.71,77.38,93.13,88.46,DISTINGUISHING CHARACTERISTICS This is an entr...,DISTINGUISHING CHARACTERISTICS. This is an adv...,3245,0,1094664,0
3,3308-915521,3308,915521,Nurse Practitioner,Nurse Practitioner - OB,168,To perform physical screening assessments; to ...,To perform physical screening assessments; to ...,91.67,84.6,96.04,85.27,and to plan patient follow-up care. SUPERVISIO...,and to plan patient follow-up care for patient...,3308,0,915521,0
4,54862-55548,54862,55548,Nurse Practitioner III,Nurse Practitioner I*,1204,"Under general direction, supervises Nurse Prac...",The Nurse Practitioner series describes levels...,82.17,80.72,81.55,89.06,"Under general direction, supervises Nurse Prac...",The Nurse Practitioner series describes levels...,54862,0,55548,0


In [ ]:
test.head()

,Sent_Id,ClassTitle,Assistant Caseworker I,Seniority_Ind,Unnamed: 4
0,1,Assistant Caseworker I,To assess the social service needs of elderly ...,0.0,NaN
1,2,Assistant Caseworker I,"and to participate in the identification, refe...",0.0,NaN
2,3,Assistant Caseworker I,DISTINGUISHING CHARACTERISTICS This is an entr...,1.0,NaN
3,5,Assistant Caseworker I,Assistant Caseworker I's are considered to be ...,1.0,NaN
4,6,Assistant Caseworker I,SUPERVISION RECEIVED AND EXERCISED Immediate s...,0.0,NaN


In [ ]:
test['Assistant Caseworker I'].nunique()

171